this notebook is to show that for certain cases, cls=0 gives exactly the same result as a regualr maskcnn.

I will use configurations in <https://github.com/leelabcnbc/thesis-yimeng-v2/blob/00b0ae57793923722941525658927954cc9c9961/results_processed/yuanyuan_8k_a_3day/data-driven_vs_transfer/baseline_comparison.ipynb>

In [1]:
# this is mostly a copy of <https://github.com/leelabcnbc/thesis-yimeng-v2/blob/master/scripts/training/yuanyuan_8k_a_3day/maskcnn_polished/master.py>
import numpy as np

from thesis_v2.data.prepared.yuanyuan_8k import get_data

from thesis_v2.training_extra.maskcnn_like.opt import get_maskcnn_v1_opt_config
from thesis_v2.training_extra.maskcnn_like.training import (train_one,
                                                            partial)

from thesis_v2.models.maskcnn_polished.builder import (
    gen_maskcnn_polished,
    load_modules as load_modules_maskcnn_polished
)


In [2]:
def train_one_maskcnn_polished(
    split_seed,
    model_seed,
    act_fn,
    loss_type,
    input_size,
    out_channel,
    num_layer,
    kernel_size_l1,
    pooling_ksize,
    scale, scale_name,
    smoothness, smoothness_name,
    pooling_type,
    bn_before_act,
    bn_after_fc
):
    
    load_modules_maskcnn_polished()
    datasets = get_data('a', 200, input_size, ('042318', '043018', '051018'), scale=0.5,
                        seed=split_seed)

    datasets = {
        'X_train': datasets[0].astype(np.float32),
        'y_train': datasets[1],
        'X_val': datasets[2].astype(np.float32),
        'y_val': datasets[3],
        'X_test': datasets[4].astype(np.float32),
        'y_test': datasets[5],
    }

    def gen_cnn_partial(input_size_cnn, n):
        return gen_maskcnn_polished(input_size=input_size_cnn,
                                    num_neuron=n,
                                    out_channel=out_channel,
                                    kernel_size_l1=kernel_size_l1,  # (try 5,9,13)
                                    kernel_size_l23=3,
                                    act_fn=act_fn,
                                    pooling_ksize=pooling_ksize,  # (try, 1,3,5,7)
                                    pooling_type=pooling_type,  # try (avg, max)  # looks that max works well here?
                                    num_layer=num_layer,
                                    bn_before_act=bn_before_act,
                                    bn_after_fc=bn_after_fc,
                                    )

    opt_config_partial = partial(get_maskcnn_v1_opt_config,
                                 scale=scale,
                                 smoothness=smoothness,
                                 group=0.0,
                                 loss_type=loss_type,
                                 )

    result = train_one(
        arch_json_partial=gen_cnn_partial,
        opt_config_partial=opt_config_partial,
        datasets=datasets,
        key=f'debug/compare_cls0_with_regular_maskcnn/regular/{model_seed}',
        show_every=100000000,
        max_epoch=40000,
        model_seed=model_seed,
        return_model=False
    )
    
    return result['stats_best']['stats']['test']['corr_mean']

In [3]:
maskcnn_param_template = {
    'out_channel': 16,
    'num_layer': 2,
    'kernel_size_l1': 9,
    'pooling_ksize': 3,
    'pooling_type': 'avg',
    'model_seed': 0,
    'split_seed': 'legacy',
}

maskcnn_param_regular = {
    **maskcnn_param_template,
    **{
        'act_fn': 'relu',
        'bn_before_act': True,
        'loss_type': 'mse',
        'smoothness': 0.000005,
        'smoothness_name': '0.000005',
        'scale': 0.01,
        'scale_name': '0.01',
        'bn_after_fc': False,
        'input_size': 50,
    }
}

print(train_one_maskcnn_polished(**maskcnn_param_regular))

num_param 20493
========starting phase 1/3==========
val metric init {'loss': 0.14660925567150115, 'loss_no_reg': 0.14377041161060333, 'corr': [0.031198956072330475, -0.028956549242138863, -0.0837211012840271, 0.04357321932911873, 0.010827144607901573, 0.0346023328602314, 0.013515481725335121, -0.046275485306978226, 0.05008558928966522, -0.006273146718740463, -0.05291634052991867, -0.04240459203720093, 0.10408493131399155, 0.15733128786087036, 0.030820943415164948, 0.10673636943101883, -0.043763935565948486, 0.09467260539531708, 0.031703561544418335, -0.022611182183027267, 0.09689978510141373, -0.07490523159503937, 0.02363152615725994, 0.008976493030786514, 0.015267157927155495, 0.0004135906638111919, -0.0011893577175214887, -0.055661555379629135, 0.07597590982913971, 0.011360326781868935, 0.02378547564148903, 0.05902416631579399, 0.03659946843981743, 0.09861762076616287, 0.03713153675198555, -0.11668487638235092, -0.0005329936393536627, 0.07098501920700073, -0.13184714317321777, -0.14

early stopping after epoch 600 metric 0.10797938704490662
========end phase 2/3==========
========starting phase 3/3==========
for grp of sz 11, lr from 0.000333 to 0.000111
val metric init {'loss': 0.11177114844322204, 'loss_no_reg': 0.10797938704490662, 'corr': [0.2911869287490845, 0.49696803092956543, 0.6110566854476929, 0.7368093729019165, 0.5168116092681885, 0.5167156457901001, 0.5108482241630554, 0.6294921040534973, 0.6785271167755127, 0.714794933795929, 0.7006577849388123, 0.5485681891441345, 0.6127716302871704, 0.6455424427986145, 0.6579211950302124, 0.6210808157920837, 0.7061901688575745, 0.6135026216506958, 0.5932409167289734, 0.6913055181503296, 0.47745707631111145, 0.4617050886154175, 0.3465803265571594, 0.11260919272899628, 0.7682605385780334, 0.6206834316253662, 0.5645780563354492, 0.3609061539173126, 0.6422755718231201, 0.5602790117263794, 0.3665769100189209, 0.5779709815979004, 0.5257043838500977, 0.5757706165313721, 0.6280243396759033, 0.5401779413223267, 0.60901051759

In [4]:
from thesis_v2.models.maskcnn_polished_with_local_pcn.builder import (
    gen_maskcnn_polished_with_local_pcn,
    load_modules as load_modules_maskcnn_polished_with_local_pcn
)

def train_one_maskcnn_polished_with_local_pcn(
    split_seed,
    model_seed,
    act_fn,
    loss_type,
    input_size,
    out_channel,
    num_layer,
    kernel_size_l1,
    pooling_ksize,
    scale, scale_name,
    smoothness, smoothness_name,
    pooling_type,
    bn_before_act,
    bn_after_fc,
    
    
    pcn_bn,
    pcn_bn_post,
    pcn_bypass,
    pcn_cls,
    pcn_final_act,
    pcn_no_act,
    pcn_bias,
):
    
    load_modules_maskcnn_polished_with_local_pcn()
    datasets = get_data('a', 200, input_size, ('042318', '043018', '051018'), scale=0.5,
                        seed=split_seed)

    datasets = {
        'X_train': datasets[0].astype(np.float32),
        'y_train': datasets[1],
        'X_val': datasets[2].astype(np.float32),
        'y_val': datasets[3],
        'X_test': datasets[4].astype(np.float32),
        'y_test': datasets[5],
    }

    def gen_cnn_partial(input_size_cnn, n):
        return gen_maskcnn_polished_with_local_pcn(
            input_size=input_size_cnn,
            num_neuron=n,
            out_channel=out_channel,
            kernel_size_l1=kernel_size_l1,  # (try 5,9,13)
            kernel_size_l23=3,
            act_fn=act_fn,
            pooling_ksize=pooling_ksize,  # (try, 1,3,5,7)
            pooling_type=pooling_type,  # try (avg, max)  # looks that max works well here?
            num_layer=num_layer,
            bn_before_act=bn_before_act,
            bn_after_fc=bn_after_fc,


            pcn_bn=pcn_bn,
            pcn_bn_post=pcn_bn_post,
            pcn_bypass=pcn_bypass,
            pcn_cls=pcn_cls,
            pcn_final_act=pcn_final_act,
            pcn_no_act=pcn_no_act,
            pcn_b0_init=1.0,
            pcn_tied=False,
            pcn_bias=pcn_bias,
        )

    opt_config_partial = partial(get_maskcnn_v1_opt_config,
                                 scale=scale,
                                 smoothness=smoothness,
                                 group=0.0,
                                 loss_type=loss_type,
                                 )

    result = train_one(
        arch_json_partial=gen_cnn_partial,
        opt_config_partial=opt_config_partial,
        datasets=datasets,
        key=f'debug/compare_cls0_with_regular_maskcnn/local_pcn/{pcn_cls}/{model_seed}',
        show_every=100000000,
        max_epoch=40000,
        model_seed=model_seed,
        return_model=False
    )
    
    return result['stats_best']['stats']['test']['corr_mean']

In [5]:
maskcnn_param_local_pcn_cls0 = {
    **maskcnn_param_template,
    **{
        'act_fn': 'relu',
        'bn_before_act': True,
        'loss_type': 'mse',
        'smoothness': 0.000005,
        'smoothness_name': '0.000005',
        'scale': 0.01,
        'scale_name': '0.01',
        'bn_after_fc': False,
        'input_size': 50,
        'pcn_bn': True,
        'pcn_bn_post': False,
        'pcn_bypass': False,
        'pcn_cls': 0,
        'pcn_final_act': True,
        
        # this can ensure we get same stuff.
        'pcn_no_act': True,
        'pcn_bias': False,
    }
}

print(train_one_maskcnn_polished_with_local_pcn(**maskcnn_param_local_pcn_cls0))

num_param 20493
========starting phase 1/3==========
val metric init {'loss': 0.14660925567150115, 'loss_no_reg': 0.14377041161060333, 'corr': [0.031198956072330475, -0.028956549242138863, -0.0837211012840271, 0.04357321932911873, 0.010827144607901573, 0.0346023328602314, 0.013515481725335121, -0.046275485306978226, 0.05008558928966522, -0.006273146718740463, -0.05291634052991867, -0.04240459203720093, 0.10408493131399155, 0.15733128786087036, 0.030820943415164948, 0.10673636943101883, -0.043763935565948486, 0.09467260539531708, 0.031703561544418335, -0.022611182183027267, 0.09689978510141373, -0.07490523159503937, 0.02363152615725994, 0.008976493030786514, 0.015267157927155495, 0.0004135906638111919, -0.0011893577175214887, -0.055661555379629135, 0.07597590982913971, 0.011360326781868935, 0.02378547564148903, 0.05902416631579399, 0.03659946843981743, 0.09861762076616287, 0.03713153675198555, -0.11668487638235092, -0.0005329936393536627, 0.07098501920700073, -0.13184714317321777, -0.14

early stopping after epoch 600 metric 0.10797938704490662
========end phase 2/3==========
========starting phase 3/3==========
for grp of sz 11, lr from 0.000333 to 0.000111
val metric init {'loss': 0.11177114844322204, 'loss_no_reg': 0.10797938704490662, 'corr': [0.2911869287490845, 0.49696803092956543, 0.6110566854476929, 0.7368093729019165, 0.5168116092681885, 0.5167156457901001, 0.5108482241630554, 0.6294921040534973, 0.6785271167755127, 0.714794933795929, 0.7006577849388123, 0.5485681891441345, 0.6127716302871704, 0.6455424427986145, 0.6579211950302124, 0.6210808157920837, 0.7061901688575745, 0.6135026216506958, 0.5932409167289734, 0.6913055181503296, 0.47745707631111145, 0.4617050886154175, 0.3465803265571594, 0.11260919272899628, 0.7682605385780334, 0.6206834316253662, 0.5645780563354492, 0.3609061539173126, 0.6422755718231201, 0.5602790117263794, 0.3665769100189209, 0.5779709815979004, 0.5257043838500977, 0.5757706165313721, 0.6280243396759033, 0.5401779413223267, 0.60901051759

In [ ]:
# exactly match.